In [2]:
import numpy as np
import pandas as pd 
from PIL import Image
import os

import torch
import torch.nn as nn
import torchvision.transforms as transform
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from tqdm import tqdm
import random

#### set random seed

In [3]:
def setSeed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

#### transform

In [6]:
test_tfm = transform.Compose([
    transform.Resize((128, 128)),
    transform.ToTensor()
])

train_tfm = transform.Compose([
    transform.RandomRotation(40),
    transform.RandomAffine(degrees=0, translate=(0.2, 0.2), shear=0.2),
    transform.RandomHorizontalFlip(p=0.5),
    transform.Resize((224, 224)),
    transform.ToTensor(),
    transform.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#### dataset

In [18]:
class foodDataset(Dataset):

    def __init__(self, path, tfm=test_tfm):
        super(foodDataset).__init__()
        self.path = path
        self.tfm = tfm
        self.imgName = sorted([name for name in os.listdir(self.path) if name.endswith('.jpg')])
        self.imgPath = [os.path.join(self.path, name) for name in self.imgName]
     
    def __getitem__(self, idx):
        img = Image.open(self.imgPath[idx])
        img = self.tfm(img)
        try:
            label = int(self.imgName[idx].split('_')[0])
        except:
            label = -1
        return img, label
    
    def __len__(self):
        return len(self.imgName)


#### model

In [19]:
class cnnBlock(nn.Module):

    def __init__(self, input_chann, output_channel, kernel_size=3, stride=1, padding=1):
        super(cnnBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(input_chann, output_channel, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(output_channel),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)
        )

    def forward(self, x):
        return self.block(x)

In [20]:
class linearBlock(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(linearBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.Dropout(0.4),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.block(x)

In [21]:
class foodClassifier(nn.Module):
    
    def __init__(self, cnnLayers, linearLayers):
        super(foodClassifier, self).__init__()
        self.cnn = nn.Sequential(
            *[cnnBlock(cnnLayers[i-1], cnnLayers[i]) for i in range(1, len(cnnLayers))]
        )
        self.linear = nn.Sequential(
            *[linearBlock(linearLayers[i-1], linearLayers[i]) for i in range(1, len(linearLayers))]
        )
    
    def forward(self, x):
        x = self.cnn(x)
        x = x.flatten(start_dim=1)
        x = self.linear(x)
        return x
        

#### Trainer

In [22]:
def Trainer(model, train_loader, valid_loader, config):
    model.to(config['device'])

    optimizer = torch.optim.Adam(model.parameters(), config['learning_rate'], weight_decay=config['weight_decay'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=config['gamma'])
    criterion = torch.nn.CrossEntropyLoss()

    best_acc = 0
    step = 0

    writer = SummaryWriter()

    for epoch in range(config['epochNum']):
        model.train()

        train_loss, train_acc = 0.0, 0.0

        for data in tqdm(train_loader):
            img, label = data
            img, label = img.to(config['device']), label.to(config['device'])

            pred = model(img)
            loss = criterion(pred, label)

            # clear gradients in every batch
            optimizer.zero_grad()

            # grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            loss.backward()

            optimizer.step()

            train_acc += (pred.argmax(dim=-1) == label).float().mean()
            train_loss += loss.item()

        train_loss = train_loss / len(train_loader)
        train_acc = train_acc / len(train_loader)
        writer.add_scalar(config['writerName'] + 'Loss/Train', train_loss, epoch)
        writer.add_scalar(config['writerName'] + 'Accuracy/Train', train_acc, epoch)

        model.eval()
        valid_acc, valid_loss = 0.0, 0.0
        for data in tqdm(valid_loader):
            img, label = data 
            img, label = img.to(config['device']), label.to(config['device'])
    
            with torch.no_grad():
                pred = model(img)

            loss = criterion(pred, label)

            valid_loss += loss.item()
            valid_acc += (pred.argmax(dim=-1) == label).float().mean()

        valid_acc, valid_loss = valid_acc / len(valid_loader), valid_loss / len(valid_loader)
        writer.add_scalar(config['writerName'] + 'Loss/Valid', valid_loss, epoch)
        writer.add_scalar(config['writerName'] + 'Accuracy/Valid', valid_acc, epoch)
        print(f'epoch[{epoch}] | train loss: {train_loss:.5f} train acc: {train_acc:.4f} | valid loss: {valid_loss:.5f} valid acc: {valid_acc:.4f}')

        scheduler.step()

        torch.save(model.state_dict(), os.path.join(config['modelSavePath'], config['lastModel']))
        if best_acc < valid_acc:
            best_acc = valid_acc
            torch.save(model.state_dict(), os.path.join(config['modelSavePath'], config['bestModel']))
            print(f'find A better model! acc: {best_acc:.4f}')
            step = 0
        else:
            step += 1
            if step > config['early_stop']:
                print('Cannot improve model~')
                break
    writer.close()


#### parameters

In [23]:
config = {
    'learning_rate': 1e-3,
    'batch_size': 64,
    'cnnLayers': [3, 64, 128, 256, 512],
    'linearLayers': [512*8*8, 1024, 512, 256, 11],
    'gamma': 0.8,
    'step_size': 10,
    'weight_decay': 1e-3,
    'seed': 914122,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'epochNum': 400,
    'early_stop': 50,
    'modelSavePath': './model/',
    'bestModel': 'best_0915.ckpt',
    'lastModel': 'last_0915.ckpt',
    'trainImgPath': './data/training/',
    'validImgPath': './data/validation/',
    'testImgPath': './data/test/',
    'writerName': 'HW3 '
}

In [1]:
print(config['device'])

NameError: name 'config' is not defined

#### data & model

In [12]:
train_set = foodDataset(config['trainImgPath'])
valid_set = foodDataset(config['validImgPath'])
test_set = foodDataset(config['testImgPath'])

FileNotFoundError: [Errno 2] No such file or directory: './data/training/'

In [13]:
train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=config['batch_size'])
test_loader = DataLoader(test_set, batch_size=config['batch_size'])

NameError: name 'train_set' is not defined

In [25]:
model = foodClassifier(config['cnnLayers'], config['linearLayers'])
# model.load_state_dict(torch.load('./model/last_0915.ckpt'))

In [26]:
Trainer(model, train_loader, valid_loader, config)

100%|██████████| 54/54 [00:14<00:00,  3.78it/s]


epoch[0] | train loss: 2.34272 train acc: 0.2039 | valid loss: 2.10553 valid acc: 0.3147
find A better model! acc: 0.3147


100%|██████████| 54/54 [00:14<00:00,  3.81it/s]


epoch[1] | train loss: 2.17223 train acc: 0.2734 | valid loss: 2.01420 valid acc: 0.3677
find A better model! acc: 0.3677


100%|██████████| 54/54 [00:13<00:00,  3.95it/s]


...

100%|██████████| 54/54 [00:14<00:00,  3.60it/s]


epoch[40] | train loss: 1.14088 train acc: 0.5961 | valid loss: 1.32349 valid acc: 0.6283


100%|██████████| 54/54 [00:13<00:00,  3.95it/s]


epoch[41] | train loss: 1.11558 train acc: 0.6017 | valid loss: 1.39495 valid acc: 0.6001


 83%|████████▎ | 128/155 [00:42<00:15,  1.75it/s]